Oppgave 1.7: Hvor mange elbiler ble førstegangsregistrert i 2022
    Leser fra preppet parquet fil laget i oppg. 1.6

In [44]:
import polars as pl

kp = pl.read_parquet('kjoretoy_preppet.parquet')

df_ant_el_2022 = kp.filter(
    (pl.col('tekn_reg_f_g_n').cast(str).str.contains('2022')) 
    & (pl.col('Er_Elbil') == True)
)

print(f"Antall Elbiler registrert i 2022 er: {df_ant_el_2022.height}")

Antall Elbiler registrert i 2022 er: 37249


Oppgave 1.8: Hvor mange prosent av persbilene solgt i 2022 var elbiler
    Her leser vi fra tekn_reg_eier_dato og sjekker at datoen starter med året 2022, da det er slik dataen er strukturert.

In [42]:
antallElbil = len(kp.filter((pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')) & (pl.col('Er_Elbil') == True)))

antallBilerTotalt = len(kp.filter(pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')))

prosent =((antallElbil / antallBilerTotalt) * 100).__round__(2)

print(f"Antall Elbiler registrert på ny eier i 2022: {antallElbil}")
print(f"Antall alle biler registrert på ny eier i 2022: {antallBilerTotalt}")
print(f"Prosent som er elbil: {prosent}")

Antall Elbiler registrert på ny eier i 2022: 38670
Antall alle biler registrert på ny eier i 2022: 133994
Prosent som er elbil: 28.86


Oppg 1.9: Hvilken bilmodell var mest populær i 2022
    Jeg tolker denne oppgaven til å bety mest populær å kjøpe, ergo vi ser på førstegangsregistrerte biler. 
    Vi filtrerer ut alle biler fra 2022, og teller antall hendelser av modellen. Deretter sorterer vi listen etter count, synkende.
    Hjelpedatasett Kjøretøygrupper er brukt ved siden av. 

In [46]:
biler2022 = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('2022') & (pl.col('tekn_kjtgrp') == 101))

populæreBiler = biler2022.group_by("tekn_modell").count()

populæreBiler.sort(pl.col('count'), descending=True)

tekn_modell,count
str,u32
"""Model Y""",8719
"""ID.4 GTX 220 K…",4021
"""XC40""",3001
"""IONIQ5""",2890
"""ENYAQ 80X""",2799
"""Polestar 2""",2413
"""Mustang Mach-E…",2389
"""RAV4""",2009
"""E TRON 55""",1975


Oppg 1.10: Hvor mange gule kjøretøy ble solgt i mai 2022?
    Sorterer datasettet på førstegangsregistrerte biler i mai, 2022, som også er av typen personbil. 
    For gøy regner jeg ut både antall gule personbiler, og gule kjøretøy av alle typer totalt. 
    Hjelpedatasett Fargekoder er brukt ved siden av. 

In [71]:
personbiler = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('202205') & (pl.col('tekn_kjtgrp') == 101))
allekjøretøy = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('202205'))

gulebiler = len(personbiler.filter(pl.col('tekn_farge').cast(str).str.contains('09')))
gulekjøretøy = len(allekjøretøy.filter(pl.col('tekn_farge').cast(str).str.contains('09')))

print(f"Antall gule personbiler: {gulebiler}")
print(f"Antall gule kjøretøy totalt: {gulekjøretøy}")

Antall gule personbiler: 33
Antall gule kjøretøy totalt: 74


Oppg 1.11: Hvor stor andel av personbilene som selges i Norge har firehjulstrekk?
    Siden det ikke defineres noe årstall for salg, så tolker jeg at det betyr alle personbilene noen gang registrert i Norge.
    Vi filtrerer først ut alle personbiler, og deretter ut alle som ikke har 2 aksler i drift. 

In [80]:
personbilerAlle = kp.filter(pl.col('tekn_kjtgrp') == 101)

firehjulstrekk = len(personbilerAlle.filter(pl.col('tekn_aksler_drift') == 2))

print(f"Antall biler solgt totalt i Norge med firehjulstrekk: {firehjulstrekk}")

Antall biler solgt totalt i Norge med firehjulstrekk: 192688


Oppg 1.12: Hvilken måned i året førstegangsregistreres det flest biler i Norge?
    I denne oppgaven valgte vi å returnere en tabell over alle 12 måneder som viser antall registrerte biler totalt i datasett i denne måneden. 
    Siden det ikke defineres noe spesielt kjøretøy, valgte vi å definere "bil" som alt i hjelpedatasettet Kjøretøygrupper som inneholder ordet "bil". (koder: 101-381)
    Vi omformer dato til Date, og henter ut måneden. Vi filtrerer ut alle kjøretøy som har gruppe lik eller mindre enn 381. Deretter grupperer vi på måneden, og teller antall som inneholder den måneden. Deretter sorterer vi på count, slik at de mest populære månedene kommer først. 
    Dette gjelder for alle år sammenlagt.
    

In [117]:
dato = kp.with_columns(
    måned = pl.col('tekn_reg_f_g_n').cast(str).str.strptime(pl.Date, format='%Y %m %d', strict=False).dt.month(), 
)

verifiserteBiler = dato.filter(pl.col('tekn_kjtgrp') <= 381)

verifiserteBiler.group_by('måned').count().sort('count', descending=True)

måned,count
i8,u32
12,57978
6,41440
3,40727
9,38833
11,37217
8,35557
5,31839
10,31400
4,27959


Oppg 1.13: Hva var den mest populære fargen for førstegangsregistrerte biler i Norge i hvert år?
    

In [155]:
alleBiler = kp.with_columns(
    år = pl.col('tekn_reg_f_g_n').cast(str).str.strptime(pl.Date, format='%Y %m %d', strict=False).dt.year(),
)

filtrerteBiler = alleBiler.filter(pl.col('tekn_kjtgrp') <= 381)

telteFarger = filtrerteBiler.select('år', '')

SchemaError: invalid series dtype: expected `Boolean`, got `u32`